In [24]:
import numpy as np
import matplotlib.pyplot as plt 
import os 
import cv2
import random
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [25]:
data_url = r"D:\Final\Machine\dataset\Dataset"
categories = ["0","1","2","3","4","5","6","7","8","9"]
dataset = []
out = []
# store and normalize data 
for cat in categories :
    path = os.path.join(data_url,cat)
    for imgPath in os.listdir(path):
        image = cv2.imread(os.path.join(path,imgPath),cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (64,64))
        image = image.astype('float32')
        image /= 255
        dataset.append(image)
        out.append(categories.index(cat))
        #plt.imshow(image,cmap="gray")
        #plt.show()
        

In [26]:
x, x_Test, y, y_Test= train_test_split(dataset, out, test_size=0.2, random_state=42)


x = np.asarray(x)
y = np.asarray(y)
x_Test= np.asarray(x_Test)
y_Test= np.asarray(y_Test)


x = x.reshape(x.shape[0],64,64,1)
x_Test= x_Test.reshape(-1,64,64,1)


y = to_categorical(y, 10)
#y_test = to_categorical(y_test, 10)
y_Test= to_categorical(y_Test,10)

# print(y_train.shape)
# plt.imshow(x[9],cmap="gray")
# plt.show()


In [27]:
model = Sequential([
    Flatten(),
    Dense(128, activation='relu'), # hidden layer 1 with 128 neurons and input layer 
    Dense(64, activation='relu'),# hidden layer 2 with 64 neurons
    Dense(32, activation='relu'),  # hidden layer 3 with 32 neurons
    Dense(10, activation='softmax'),  # out layer 
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#hist = model.fit(x_train, y_train,batch_size=32, epochs=10,validation_data=(x_valid, y_valid))

In [28]:
# k-fold cross-validation during trianing
scores, histories = list(), list()
sum_ = 0.0
# prepare cross validation
K=6
kfold = KFold(K)

# enumerate splits
for train_index, valid_index in kfold.split(x):
    # select rows for train and test
    x_train, x_valid = x[train_index], x[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    # fit model
    history = model.fit(x_train, y_train, epochs=15, batch_size=32, validation_data=(x_valid, y_valid), verbose=0)
    # evaluate model
    _, acc = model.evaluate(x_valid,y_valid, verbose=0)
    # stores scores
    scores.append(acc)
    histories.append(history)
    sum_ += acc
scoreOfVal = sum_ / K

#Testing Accuracy
_, scoreOfTest = model.evaluate(x_Test, y_Test, batch_size=32, verbose=0)


In [29]:

print('accuracy for Validation = ', scoreOfVal)
print('accuracy for Testing = ', scoreOfTest)

y_pred = model.predict(x_Test)
# print(y_pred)


y_pred_ =np.argmax(y_pred, axis=1)
y_test_ =np.argmax(y_Test, axis=1)
# print(y_test)


print(classification_report(y_test_, y_pred_))
print(confusion_matrix(y_test_, y_pred_))

# # Precision= TP/(TP + FP) = Accuracy of positive predictions.

# # Recall = TP/(TP+FN) = Fraction of positives that were correctly identified.

# # F1 Score = 2*(Recall * Precision) / (Recall + Precision) = What percent of positive predictions were correct? 



accuracy for Validation =  0.7829573154449463
accuracy for Testing =  0.7312348484992981
13/13 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.89      0.78      0.83        40
           1       0.93      0.81      0.87        48
           2       0.68      0.86      0.76        50
           3       0.72      0.80      0.76        35
           4       0.52      0.78      0.62        32
           5       0.69      0.92      0.79        38
           6       0.67      0.80      0.73        41
           7       0.77      0.53      0.62        38
           8       0.83      0.42      0.56        45
           9       0.78      0.63      0.70        46

    accuracy                           0.73       413
   macro avg       0.75      0.73      0.72       413
weighted avg       0.76      0.73      0.73       413

[[31  0  1  1  4  0  2  0  0  1]
 [ 1 39  4  2  0  1  1  0  0  0]
 [ 0  1 43  1  1  0  3  1  0  0]
